# Customer RFM Analysis
Analysis for shipping cost voucher receiver candidate

In [1]:
import pandas as pd

# import postgres connection factory utility
from util.dbconn import DbConn

In [2]:
WAREHOUSE_CONN = {
    'user': 'postgres',
    'password': 'qshOke46RvOg0',
    'host': 'localhost',
    'port': '8084',
    'dbname': 'warehouse'
}

In [3]:
postgres_conn = DbConn(WAREHOUSE_CONN).get_conn()

In [4]:
customer_recency_query = "SELECT \
    user_name, \
    last_order_interval,  \
    EXTRACT (epoch from last_order_interval/3600) AS hour_since_last_order \
FROM \
    mart_customer_recency \
; \
"
customer_recency = pd.read_sql(customer_recency_query, con=postgres_conn)
display(customer_recency.head(5))
display(customer_recency.describe())

,user_name,last_order_interval,hour_since_last_order
0,8cdbbdb32c682663f17135569a85c24f,383 days 02:29:57,9194.499167
1,b3ab1572fdb4a1c2e6862a3ea20e68dc,287 days 02:56:37,6890.943611
2,ba4a42117e2f671a90e2a16844cb28f2,429 days 00:30:41,10296.511389
3,10b6d571eedd3e1bbf10521ee704dcd4,246 days 22:53:32,5926.892222
4,ed91e29ad9cf4fad5935469b0e91e792,129 days 04:53:13,3100.886944


,last_order_interval,hour_since_last_order
count,94986,94986.000000
mean,288 days 10:34:55.245594096,6922.582013
std,153 days 00:02:46.167522262,3672.046158
min,45 days 08:23:21,1088.389167
25%,164 days 07:22:31.500000,3943.375417
50%,269 days 03:14:47,6459.246389
75%,397 days 07:18:33.500000,9535.309306
max,773 days 20:14:59,18572.249722


In [5]:
customer_frequency_query = "SELECT \
    user_name, \
    order_count, \
    avg_time_between_order, \
    EXTRACT (epoch from avg_time_between_order/3600) AS avg_hour_between_orders \
FROM \
    mart_customer_order_frequency"
customer_frequency = pd.read_sql(customer_frequency_query, con=postgres_conn)
display(customer_frequency.head(5))
display(customer_frequency.describe())

,user_name,order_count,avg_time_between_order,avg_hour_between_orders
0,0000366f3b9a7992bf8c76cfdf3221e2,1,0 days,0.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,0 days,0.0
2,0000f46a3911fa3c0805444483337064,1,0 days,0.0
3,0000f6ccb0745a6a4b88665a16c9f078,1,0 days,0.0
4,0004aac84e0df4da2b147fca70cf8255,1,0 days,0.0


,order_count,avg_time_between_order,avg_hour_between_orders
count,94986.000000,94986,94986.000000
mean,1.033858,2 days 10:43:51.761414313,58.731045
std,0.210807,23 days 08:52:28.579968842,560.874606
min,1.000000,0 days 00:00:00,0.000000
25%,1.000000,0 days 00:00:00,0.000000
50%,1.000000,0 days 00:00:00,0.000000
75%,1.000000,0 days 00:00:00,0.000000
max,16.000000,608 days 23:29:38,14615.493889


In [6]:
customer_mv_query = "SELECT * FROM fact_customer_monetary_value"
customer_monetary_value = pd.read_sql(customer_mv_query, con=postgres_conn)
display(customer_monetary_value.head(5))
display(customer_monetary_value.describe())

,state_id,state,user_sk,order_date,avg_spending,highest_spending,lowest_spending,avg_shipping_cost_vs_price_ratio
0,1.0,ACEH,1.0,20170424.0,297140.0,297140.0,297140.0,0.104610
1,1.0,ACEH,1.0,NaN,297140.0,297140.0,297140.0,0.104610
2,1.0,ACEH,2.0,20170522.0,130560.0,130560.0,130560.0,0.135304
3,1.0,ACEH,2.0,NaN,130560.0,130560.0,130560.0,0.135304
4,1.0,ACEH,3.0,20180129.0,237000.0,237000.0,237000.0,0.102326


,state_id,user_sk,order_date,avg_spending,highest_spending,lowest_spending,avg_shipping_cost_vs_price_ratio
count,192292.000000,192258.000000,9.727200e+04,1.922870e+05,1.922870e+05,1.922870e+05,192287.000000
mean,13.048239,48061.389908,2.017605e+07,1.605895e+05,1.621959e+05,1.596615e+05,0.308953
std,9.229428,27737.044017,4.870463e+03,2.194887e+05,2.295727e+05,2.191479e+05,0.315183
min,1.000000,1.000000,2.016090e+07,9.590000e+03,9.590000e+03,9.590000e+03,0.000000
25%,7.000000,24049.000000,2.017091e+07,6.225000e+04,6.243000e+04,6.169000e+04,0.132120
50%,10.000000,48071.500000,2.018012e+07,1.056500e+05,1.061700e+05,1.047900e+05,0.224680
75%,16.000000,72083.000000,2.018050e+07,1.768950e+05,1.779900e+05,1.758800e+05,0.381619
max,34.000000,96096.000000,2.018090e+07,1.366408e+07,1.366408e+07,1.366408e+07,21.447059


In [7]:
customer_rmf_query = """WITH all_user AS (
SELECT
    DISTINCT od.user_name,
    ud.user_sk,
    ud.customer_state_id AS state_id,
    ud.customer_city AS city
FROM
    order_dim od
LEFT JOIN user_dim ud ON
    od.user_name = ud.user_name),
distinct_fc_monev AS (
SELECT
    *
FROM
    fact_customer_monetary_value fcmv2
WHERE
    user_sk IS NOT NULL
    AND order_date IS NULL )
SELECT
    u.user_name,
    u.state_id,
    u.city,
    kbll.lat,
    kbll.long,
    fcmv.avg_spending,
    mcof.avg_time_between_order,
    mcof.order_count,
    mcr.last_order_interval
FROM
    all_user u
LEFT JOIN distinct_fc_monev fcmv ON
    u.user_sk = fcmv.user_sk
LEFT JOIN mart_customer_order_frequency mcof ON
    u.user_name = mcof.user_name
LEFT JOIN mart_customer_recency mcr ON
    u.user_name = mcr.user_name
LEFT JOIN ext_lat_long_kabko_id kbll ON
    lower(trim(u.city)) = lower(trim(kbll.kabko));"""
customer_rmf = pd.read_sql(customer_rmf_query, con=postgres_conn)
display(customer_rmf.info())
display(customer_rmf.head(5))
display(customer_rmf.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96096 entries, 0 to 96095
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   user_name               96096 non-null  object         
 1   state_id                96096 non-null  int64          
 2   city                    96096 non-null  object         
 3   lat                     82335 non-null  float64        
 4   long                    82335 non-null  float64        
 5   avg_spending            94983 non-null  float64        
 6   avg_time_between_order  94986 non-null  timedelta64[ns]
 7   order_count             94986 non-null  float64        
 8   last_order_interval     94986 non-null  timedelta64[ns]
dtypes: float64(4), int64(1), object(2), timedelta64[ns](2)
memory usage: 6.6+ MB


None

,user_name,state_id,city,lat,long,avg_spending,avg_time_between_order,order_count,last_order_interval
0,349eebf90d7966610f93da0df77c304b,1,KABUPATEN ACEH BARAT,4.4543,96.1527,233810.0,0 days 00:00:00,1.0,450 days 01:24:25
1,ee8f40b7a6867725cbf3064943ba9eeb,1,KABUPATEN ACEH BARAT,4.4543,96.1527,83700.0,0 days 00:00:00,1.0,267 days 06:29:18
2,65afec511846f28c3a848b24df189761,1,KABUPATEN ACEH BARAT,4.4543,96.1527,45310.0,0 days 00:00:00,1.0,231 days 06:08:04
3,cc759acc27c23ac89d82ea1bb8fea2e9,1,KABUPATEN ACEH BARAT,4.4543,96.1527,79040.0,0 days 00:00:00,1.0,78 days 04:28:28
4,6d74740d498504341df1e7f104daebf0,1,KABUPATEN ACEH BARAT,4.4543,96.1527,102460.0,188 days 04:29:01,2.0,275 days 19:30:52


,state_id,lat,long,avg_spending,avg_time_between_order,order_count,last_order_interval
count,96096.000000,82335.000000,82335.000000,9.498300e+04,94986,94986.000000,94986
mean,13.044206,-4.926417,110.217252,1.607270e+05,2 days 10:43:51.761414313,1.033858,288 days 10:34:55.245594088
std,9.229872,3.204223,7.560268,2.198878e+05,23 days 08:52:28.579968842,0.210807,153 days 00:02:46.167522262
min,1.000000,-10.738600,95.323800,9.590000e+03,0 days 00:00:00,1.000000,45 days 08:23:21
25%,7.000000,-7.005100,106.652700,6.241000e+04,0 days 00:00:00,1.000000,164 days 07:22:31.500000
50%,10.000000,-6.202400,107.541300,1.057400e+05,0 days 00:00:00,1.000000,269 days 03:14:47
75%,16.000000,-3.320000,112.667500,1.769650e+05,0 days 00:00:00,1.000000,397 days 07:18:33.500000
max,34.000000,5.892600,140.762400,1.366408e+07,608 days 23:29:38,16.000000,773 days 20:14:59
